In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Supervised Fine Tuning with Gemini 1.5 Flash for Q&A

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/supervised_finetuning_using_gemini_qa.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Ftuning%2Fsupervised_finetuning_using_gemini_qa.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/tuning/supervised_finetuning_using_gemini_qa.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/supervised_finetuning_using_gemini_qa.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Erwin Huizenga](https://github.com/erwinh85) |

## Overview

**Gemini** is a family of generative AI models developed by Google DeepMind designed for multimodal use cases. The Gemini API gives you access to the various Gemini models, such as Gemini 1.5 Pro and Gemini 1.5 Flash.
This notebook demonstrates fine-tuning the Gemini 1.5 Flahs using the Vertex AI Supervised Tuning feature. Supervised Tuning allows you to use your own labeled training data to further refine the base model's capabilities toward your specific tasks.
Supervised Tuning uses labeled examples to tune a model. Each example demonstrates the output you want from your text model during inference.
First, ensure your training data is of high quality, well-labeled, and directly relevant to the target task. This is crucial as low-quality data can adversely affect the performance and introduce bias in the fine-tuned model.
Training: Experiment with different configurations to optimize the model's performance on the target task.
Evaluation:
Metric: Choose appropriate evaluation metrics that accurately reflect the success of the fine-tuned model for your specific task
Evaluation Set: Use a separate set of data to evaluate the model's performance


Refer to public [documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini-supervised-tuning) for more details.

<hr/>

Before running this notebook, ensure you have:

- A Google Cloud project: Provide your project ID in the `PROJECT_ID` variable.

- Authenticated your Colab environment: Run the authentication code block at the beginning.

- Prepared training data (Test with your own data or use the one in the notebook): Data should be formatted in JSONL with prompts and corresponding completions.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

To get an estimate of the number of tokens

## Get started

### Install Vertex AI SDK and other required packages


In [11]:
%pip install --upgrade --user --quiet google-cloud-aiplatform datasets

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [12]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [3]:
# Use the environment variable if the user doesn't provide Project ID.
import os
import vertexai

PROJECT_ID = "<your_project_id>"  # @param {type:"string", isTemplate: true}
if PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [189]:
# Vertex AI SDK
from google.cloud import aiplatform
from google.cloud.aiplatform.metadata import context
from google.cloud.aiplatform.metadata import utils as metadata_utils
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
)
from vertexai.preview.tuning import sft

# Vertex AI SDK
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import array
import time
from datasets import load_dataset
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Data

#### SQuAD dataset
Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

You can fine more information on the SQuAD [github page](https://rajpurkar.github.io/SQuAD-explorer/)**bold text**

First update the `BUCKET_NAME` parameter below. You can either use an existing bucket or create a new one.

In [95]:
# Provide a bucket name
BUCKET_NAME = "tuning-demo-erwinh/gemini-tuning"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
print(BUCKET_URI)

gs://tuning-demo-erwinh/gemini-tuning


Only run the code below if you want to create a new Google Cloud Storage bucket.

In [ ]:
# ! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Next you will copy the data into your bucket.

In [ ]:
!gsutil cp gs://github-repo/generative-ai/gemini/tuning/qa/squad_test.csv .
!gsutil cp gs://github-repo/generative-ai/gemini/tuning/qa/squad_train.csv .
!gsutil cp gs://github-repo/generative-ai/gemini/tuning/qa/squad_validation.csv .

### Baseline

Next you will prepare some test data that you will use to establish a baseline.  This means evaluating your chosen model on a representative sample of your dataset before any fine-tuning. A baseline allows you to quantify the improvements achieved through fine-tuning.

In [181]:
test_df = pd.read_csv('squad_test.csv')
test_df.head(1)

,id,title,context,question,answers
0,56de3cd0cffd8e1900b4b6bf,Normans,"Normans came into Scotland, building castles a...",What culture's arrival in Scotland is know as ...,Norman


You will need to do some dataset preperations. We will add a system instruction to the dataset:

`SystemInstruct`: System instructions are a set of instructions that the model processes before it processes prompts. We recommend that you use system instructions to tell the model how you want it to behave and respond to prompts.

We will also combine the `context` and `question`.

In [80]:
systemInstruct = "Answer the question based on the context"

In [182]:
# combine the systeminstruct + context + question into one column.
row_dataset = 6

test_df["input_question"] = systemInstruct + "\n" + "Context: " + test_df["context"] + "\n" + "Question: " + test_df["question"]
test_question = test_df["input_question"].iloc[row_dataset]
print(test_question)

Answer the question based on the context
Context: In July 1973, as part of its outreach programme to young people, the V&A became the first museum in Britain to present a rock concert. The V&A presented a combined concert/lecture by British progressive folk-rock band Gryphon, who explored the lineage of mediaeval music and instrumentation and related how those contributed to contemporary music 500 years later. This innovative approach to bringing young people to museums was a hallmark of the directorship of Roy Strong and was subsequently emulated by some other British museums.
Question: Which musical group did the V&A present in July 1973 as part of its youth outreach programme?


Next, set the model that you will use. In this example you will use `gemini-1.5-flash-002`. A multimodal model that is designed for high-volume, cost-effective applications, and which delivers speed and efficiency to build fast, lower-cost applications that don't compromise on quality.



In [183]:
base_model = "gemini-1.5-flash-002"
generation_model = GenerativeModel(base_model)

Next lets take a question and get a prediction from Gemini that we can compare to the actual answer.

In [184]:
def get_predictions(question: str) -> str:
  """Generates predictions for a given test question.

  Args:
    test_question: The question to generate predictions for.

  Returns:
    The generated prediction text.
  """

  prompt = f"{question}"

  generation_config = GenerationConfig(
    temperature=0.1)

  response = generation_model.generate_content(
      contents=prompt, generation_config=generation_config
  ).text

  return response

In [186]:
test_answer = test_df["answers"].iloc[row_dataset]

response = get_predictions(test_question)

print(f"Gemini response: {response}")
print(f"Actual answer: {test_answer}")

Gemini response: The V&A presented the British progressive folk-rock band Gryphon.

Actual answer: Gryphon


You can see that both answers are correct, but the response from Gemini is more lengthy. However, answers in the SQuAD dataset are typically concise and clear.

Fine-tuning is a great way to control the type of output your use case requires. In this instance, you would want the model to provide short, clear answers.

Next, let's check if each dataset has an equal number of examples.

In [188]:
num_strings_pred = np.sum([isinstance(item, str) for item in y_pred])
print(f"Number of strings in y_pred: {num_strings_pred}")

num_strings_true = np.sum([isinstance(item, str) for item in y_true])
print(f"Number of strings in y_true: {num_strings_true}")

Number of strings in y_pred: 40
Number of strings in y_true: 40


Next lest establish a baseline using evaluation metrics.

Evaluating the performance of a Question Answering (QA) system requires specific metrics. Two commonly used metrics are Exact Match (EM) and F1 score.

EM is a strict measure that only considers an answer correct if it perfectly matches the ground truth, even down to the punctuation. It's a binary metric - either 1 for a perfect match or 0 otherwise. This makes it sensitive to minor variations in phrasing.

F1 score is more flexible. It considers the overlap between the predicted answer and the true answer in terms of individual words or tokens. It calculates the harmonic mean of precision (proportion of correctly predicted words out of all predicted words) and recall (proportion of correctly predicted words out of all true answer words). This allows for partial credit and is less sensitive to minor wording differences.

In practice, EM is useful when exact wording is crucial, while F1 is more suitable when evaluating the overall understanding and semantic accuracy of the QA system. Often, both metrics are used together to provide a comprehensive evaluation.

In [190]:
def calculate_em_and_f1_for_text_arrays(y_true, y_pred, average='weighted'):
    """
    Calculates the Exact Match (EM) and F1 score for arrays of text
    using word-level comparisons.

    Args:
        y_true: An array of ground truth strings.
        y_pred: An array of predicted strings.
        average: The averaging method to use for F1 score.

    Returns:
        A tuple containing the EM score and the F1 score.
    """

    em = np.mean([t == p for t, p in zip(y_true, y_pred)])

    # Use TF-IDF to convert strings to numerical vectors
    vectorizer = TfidfVectorizer()
    all_text = np.concatenate((y_true, y_pred))
    vectorizer.fit(all_text)
    y_true_vec = vectorizer.transform(y_true)
    y_pred_vec = vectorizer.transform(y_pred)

    # Calculate F1 score based on common words (non-zero elements)
    y_true_class = (y_true_vec > 0).toarray().astype(int)
    y_pred_class = (y_pred_vec > 0).toarray().astype(int)

    f1 = f1_score(y_true_class, y_pred_class, average=average)

    return em, f1


In [191]:
em, f1 = calculate_em_and_f1_for_text_arrays(y_pred, y_true)
print(f"EM score: {em}")
print(f"F1 score: {f1}")

EM score: 0.0
F1 score: 0.030862136294937427


### Prepare the data for fine-tuning

To optimize the tuning process for a foundation model, ensure your dataset includes examples that reflect the desired task. Structure your training data in a text-to-text format, where each record in the dataset pairs an input text (or prompt) with its corresponding expected output. This supervised tuning approach uses the dataset to effectively teach the model the specific behavior or task you need it to perform, by providing numerous illustrative examples.

The size of your dataset will vary depending on the complexity of the task, but as a general rule, the more examples you include, the better the model's performance.

Dataset Format
Your training data should be structured in a JSONL file and stored at a Google Cloud Storage (GCS) URI.  Each line in the JSONL file must adhere to the following schema:

A `contents` array containing objects that define:
- A `role` ("user" for user input or "model" for model output)
- `parts` containing the input data.

```
{
   "contents":[
      {
         "role":"user",  # This indicate input content
         "parts":[
            {
               "text":"How are you?"
            }
         ]
      },
      {
         "role":"model", # This indicate target content
         "parts":[ # text only
            {
               "text":"I am good, thank you!"
            }
         ]
      }
      #  ... repeat "user", "model" for multi turns.
   ]
}
```

Refer to the public [documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini-supervised-tuning-prepare#about-datasets) for more details.

In [192]:
# combine the systeminstruct + context + question into one column.
train_df = pd.read_csv('squad_train.csv')
validation_df = pd.read_csv('squad_validation.csv')

train_df["input_question"] = systemInstruct + "Context: " + train_df["context"] + "Question: " + train_df["question"]
validation_df["input_question"] = systemInstruct + "Context: " + validation_df["context"] + "Question: " + validation_df["question"]

In [197]:
def df_to_jsonl(df, output_file):
  """Converts a Pandas DataFrame to JSONL format and saves it to a file.

  Args:
    df: The DataFrame to convert.
    output_file: The name of the output file.
  """

  with open(output_file, 'w') as f:
    for row in df.itertuples(index=False):
      jsonl_obj = {
          "systemInstruction": {"parts": [{"text": "Answer the question based on the provided context."}]},
          "contents": [
              {
                  "role": "user",
                  "parts": [{"text": f"Context: {row.context}\n\nQuestion: {row.question}"}]
              },
              {"role": "model", "parts": [{"text": row.answers}]},
          ]
      }
      f.write(json.dumps(jsonl_obj) + '\n')

# Process the DataFrames
df_to_jsonl(train_df, 'squad_train.jsonl')
df_to_jsonl(validation_df, 'squad_validation.jsonl')

print(f"JSONL data written to squad_train.jsonl")
print(f"JSONL data written to squad_validation.jsonl")

JSONL data written to squad_train.jsonl
JSONL data written to squad_validation.jsonl


Next you will copy the files into your Google Cloud bucket

In [195]:
!gsutil cp ./squad_train.jsonl {BUCKET_URI}
!gsutil cp ./squad_validation.jsonl {BUCKET_URI}

Copying file://./squad_train.jsonl [Content-Type=application/octet-stream]...
/ [1 files][527.0 KiB/527.0 KiB]                                                
Operation completed over 1 objects/527.0 KiB.                                    
Copying file://./squad_validation.jsonl [Content-Type=application/octet-stream]...
/ [1 files][110.9 KiB/110.9 KiB]                                                
Operation completed over 1 objects/110.9 KiB.                                    


### Start fine-tuning job
Next you can start the fine-tuning job.

- `source_model`: Specifies the base Gemini model version you want to fine-tune.
 - `train_dataset`: Path to your training data in JSONL format.

  *Optional parameters*
 - `validation_dataset`: If provided, this data is used to evaluate the model during tuning.
 - `tuned_model_display_name`: Display name for the tuned model.
 - `epochs`: The number of training epochs to run.
 - `learning_rate_multiplier`: A value to scale the learning rate during training.
 - `adapter_size` : Gemini 1.5 Flash supports Adapter length [1, 4], default value is 4.

 **Important**: The default hyperparameter settings are optimized for optimal performance based on rigorous testing and are recommended for initial use. Users may customize these parameters to address specific performance requirements.**

In [139]:
tuned_model_display_name = "erwinh-fine-tuning-flash"  # @param {type:"string"}

sft_tuning_job = sft.train(
    source_model=base_model,
    train_dataset=f"""{BUCKET_URI}/squad_train.jsonl""",
    # # Optional:
    validation_dataset=f"""{BUCKET_URI}/squad_validation.jsonl""",
    tuned_model_display_name=tuned_model_display_name,
)

INFO:vertexai.tuning._tuning:Creating SupervisedTuningJob
INFO:vertexai.tuning._tuning:SupervisedTuningJob created. Resource name: projects/713601331534/locations/us-central1/tuningJobs/8356726629560483840
INFO:vertexai.tuning._tuning:To use this SupervisedTuningJob in another session:
INFO:vertexai.tuning._tuning:tuning_job = sft.SupervisedTuningJob('projects/713601331534/locations/us-central1/tuningJobs/8356726629560483840')
INFO:vertexai.tuning._tuning:View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/8356726629560483840?project=713601331534


In [198]:
# Get the tuning job info.
sft_tuning_job.to_dict()

{'name': 'projects/713601331534/locations/us-central1/tuningJobs/8356726629560483840',
 'tunedModelDisplayName': 'erwinh-fine-tuning-flash',
 'baseModel': 'gemini-1.5-flash-002',
 'supervisedTuningSpec': {'trainingDatasetUri': 'gs://tuning-demo-erwinh/gemini-tuning/squad_train.jsonl',
  'validationDatasetUri': 'gs://tuning-demo-erwinh/gemini-tuning/squad_validation.jsonl',
  'hyperParameters': {'epochCount': '10',
   'learningRateMultiplier': 1.0,
   'adapterSize': 'ADAPTER_SIZE_EIGHT'}},
 'state': 'JOB_STATE_SUCCEEDED',
 'createTime': '2024-10-09T06:20:11.698883Z',
 'startTime': '2024-10-09T06:20:11.740358Z',
 'endTime': '2024-10-09T06:32:38.867719Z',
 'updateTime': '2024-10-09T06:32:38.867719Z',
 'experiment': 'projects/713601331534/locations/us-central1/metadataStores/default/contexts/tuning-experiment-20241008232013040864',
 'tunedModel': {'model': 'projects/713601331534/locations/us-central1/models/1582035604160380928@1',
  'endpoint': 'projects/713601331534/locations/us-central1/

In [199]:
# Get the resource name of the tuning job
sft_tuning_job_name = sft_tuning_job.resource_name
sft_tuning_job_name

'projects/713601331534/locations/us-central1/tuningJobs/8356726629560483840'

**Important:** Tuning time depends on several factors, such as training data size, number of epochs, learning rate multiplier, etc.

<div class="alert alert-block alert-warning">
<b>⚠️ It will take ~30 mins for the model tuning job to complete on the provided dataset and set configurations/hyperparameters. ⚠️</b>
</div>

In [200]:
%%time
# Wait for job completion
while not sft_tuning_job.refresh().has_ended:
    time.sleep(60)

CPU times: user 67.5 ms, sys: 4.98 ms, total: 72.5 ms
Wall time: 1.28 s


In [201]:
# tuned model name
tuned_model_name = sft_tuning_job.tuned_model_name
tuned_model_name

'projects/713601331534/locations/us-central1/models/1582035604160380928@1'

In [202]:
# tuned model endpoint name
tuned_model_endpoint_name = sft_tuning_job.tuned_model_endpoint_name
tuned_model_endpoint_name

'projects/713601331534/locations/us-central1/endpoints/5693131570647400448'

#### Model tuning metrics

- `/train_total_loss`: Loss for the tuning dataset at a training step.
- `/train_fraction_of_correct_next_step_preds`: The token accuracy at a training step. A single prediction consists of a sequence of tokens. This metric measures the accuracy of the predicted tokens when compared to the ground truth in the tuning dataset.
- `/train_num_predictions`: Number of predicted tokens at a training step

#### Model evaluation metrics:

- `/eval_total_loss`: Loss for the evaluation dataset at an evaluation step.
- `/eval_fraction_of_correct_next_step_preds`: The token accuracy at an evaluation step. A single prediction consists of a sequence of tokens. This metric measures the accuracy of the predicted tokens when compared to the ground truth in the evaluation dataset.
- `/eval_num_predictions`: Number of predicted tokens at an evaluation step.

The metrics visualizations are available after the model tuning job completes. If you don't specify a validation dataset when you create the tuning job, only the visualizations for the tuning metrics are available.

In [155]:
# Locate Vertex AI Experiment and Vertex AI Experiment Run
experiment = aiplatform.Experiment(experiment_name=experiment_name)
filter_str = metadata_utils._make_filter_string(
    schema_title="system.ExperimentRun",
    parent_contexts=[experiment.resource_name],
)
experiment_run = context.Context.list(filter_str)[0]

In [156]:
# Read data from Tensorboard
tensorboard_run_name = f"{experiment.get_backing_tensorboard_resource().resource_name}/experiments/{experiment.name}/runs/{experiment_run.name.replace(experiment.name, '')[1:]}"
tensorboard_run = aiplatform.TensorboardRun(tensorboard_run_name)
metrics = tensorboard_run.read_time_series_data()

In [157]:
def get_metrics(metric: str = "/train_total_loss"):
    """
    Get metrics from Tensorboard.

    Args:
      metric: metric name, eg. /train_total_loss or /eval_total_loss.
    Returns:
      steps: list of steps.
      steps_loss: list of loss values.
    """
    loss_values = metrics[metric].values
    steps_loss = []
    steps = []
    for loss in loss_values:
        steps_loss.append(loss.scalar.value)
        steps.append(loss.step)
    return steps, steps_loss

In [158]:
# Get Train and Eval Loss
train_loss = get_metrics(metric="/train_total_loss")
eval_loss = get_metrics(metric="/eval_total_loss")

In [161]:
# Plot the train and eval loss metrics using Plotly python library
fig = make_subplots(
    rows=1, cols=2, shared_xaxes=True, subplot_titles=("Train Loss", "Eval Loss")
)

# Add traces
fig.add_trace(
    go.Scatter(x=train_loss[0], y=train_loss[1], name="Train Loss", mode="lines"),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(x=eval_loss[0], y=eval_loss[1], name="Eval Loss", mode="lines"),
    row=1,
    col=2,
)

# Add figure title
fig.update_layout(title="Train and Eval Loss", xaxis_title="Steps", yaxis_title="Loss")

# Set x-axis title
fig.update_xaxes(title_text="Steps")

# Set y-axes titles
fig.update_yaxes(title_text="Loss")

# Show plot
fig.show()

In [162]:
if True:
    tuned_genai_model = GenerativeModel(tuned_model_endpoint_name)
    # Test with the loaded model.
    print("***Testing***")
    print(
        tuned_genai_model.generate_content(
            contents=prompt, generation_config=generation_config
        )
    )
else:
    print("State:", sft_tuning_job.state)
    print("Error:", sft_tuning_job.error)

***Testing***
candidates {
  content {
    role: "model"
    parts {
      text: "European Council\n\n"
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.11596920092900594
}
usage_metadata {
  prompt_token_count: 290
  candidates_token_count: 3
  total_token_count: 293
}



### Model usage and evaluation.

Next you will evaluate the model to see how well it performs. You can also compare it to the benchmark.

In [164]:
y_true = test_df["answers"].values

def get_predictions(test_question):

  prompt = f"""{test_question}"""

  generation_config = GenerationConfig(
      temperature=0.1,
  )

  response = tuned_genai_model.generate_content(contents=prompt, generation_config=generation_config).text

  return response

In [165]:
y_pred = []
y_pred_question = test_df["question"].values

for i in y_pred_question:

  prediction = get_predictions(i)
  y_pred.append(prediction)

In [166]:
em, f1 = calculate_em_and_f1_for_text_arrays(y_pred, y_true)
print(f"EM score: {em}")
print(f"F1 score: {f1}")

EM score: 0.0
F1 score: 0.2399679487179487
